# Análise de Gastos com Passagens Aéreas (SCDP)

## Informações
* Origem dos dados
```
Portal da Transparência (http://transparencia.gov.br/download-de-dados/viagens)
```
* Dicionário de dados
```
http://transparencia.gov.br/pagina-interna/603364-dicion%C3%A1rio-de-dados-viagens-a-Servi%C3%A7o-Pagamentos
```
* 4 arquivos:
    * 2018_Pagamento.csv
    * 2018_Passagem.csv
    * 2018_Trecho.csv
    * 2018_Viagem.csv

## Perguntas a serem respondidas?

* Qual o trecho mais executado?
* Qual o valor médio para o trecho mais executado?
* Qual a mediana para o trecho mais executado?
* Identificar possíveis outliers nos valores do trecho mais executado

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

plt.rcParams['figure.dpi'] = 90

# Leitura dos Dados
---

In [2]:
"""
Leia o arquivo
'dados/c04_passagens/2018_Passagem.csv.zip'
"""
df_passagens = pd.read_csv('dados/c04_passagens/2018_Passagem.csv.zip', encoding='latin1', sep=';')

# Identificação de valores nulos ou faltantes
---

In [3]:
df_passagens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310816 entries, 0 to 310815
Data columns (total 16 columns):
Identificador do processo de viagem    310816 non-null int64
Meio de transporte                     310816 non-null object
País - Origem ida                      310816 non-null object
UF - Origem ida                        303560 non-null object
Cidade - Origem ida                    310816 non-null object
País - Destino ida                     310816 non-null object
UF - Destino ida                       301096 non-null object
Cidade - Destino ida                   310816 non-null object
País - Origem volta                    310816 non-null object
UF - Origem volta                      307823 non-null object
Cidade - Origem volta                  310816 non-null object
Pais - Destino volta                   310816 non-null object
UF - Destino volta                     310234 non-null object
Cidade - Destino volta                 310816 non-null object
Valor da passagem     

In [4]:
df_passagens.head()

,Identificador do processo de viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço
0,14046485,Rodoviário,Brasil,Ceará,Fortaleza,Brasil,Ceará,Sobral,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,"40,95","13,51"
1,14046485,Rodoviário,Brasil,Ceará,Sobral,Brasil,Ceará,Fortaleza,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,"33,25","10,97"
2,14166390,Aéreo,Brasil,Paraná,Londrina,Brasil,Mato Grosso,Sinop,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,"518,09",",00"
3,14166390,Aéreo,Brasil,Mato Grosso,Sinop,Brasil,Paraná,Londrina,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,"665,38",",00"
4,14201345,Aéreo,Brasil,Paraná,Curitiba,Brasil,Rio Grande do Sul,Porto Alegre,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,"369,30",",00"


In [5]:
"""
Vamos verificar algumas estatísticas básicas sobre colunas numéricas.
"""

df_passagens.describe()

,Identificador do processo de viagem
count,310816.00
mean,34950773.58
std,199220763.01
min,14046485.00
25%,14763521.75
50%,14929268.00
75%,15105045.00
max,2018002771.00


In [6]:
"""
Vamos verificar algumas estatísticas básicas sobre as demais colunas.
"""

df_passagens.describe(include='object')

,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço
count,310816,310816,303560,310816,310816,301096,310816,310816,307823,310816,310816,310234,310816,310816,310816
unique,4,163,28,913,162,28,997,111,28,541,103,28,352,72794,1641
top,Aéreo,Brasil,Distrito Federal,Brasília,Brasil,Distrito Federal,Brasília,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,Sem Informação,",00",",00"
freq,293475,303557,80478,80478,301093,79346,79346,287331,287331,287331,287331,287331,287331,3299,281439


In [7]:
"""
Tratamento em todas as colunas para colocar o valor None quando o valor for 'Não informado' ou NaN
"""
for c in df_passagens.columns:
    df_passagens[c] = df_passagens[c].apply(lambda x: x if x != 'Sem Informação' and pd.notnull(x) else None)

In [8]:
df_passagens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310816 entries, 0 to 310815
Data columns (total 16 columns):
Identificador do processo de viagem    310816 non-null int64
Meio de transporte                     310816 non-null object
País - Origem ida                      310816 non-null object
UF - Origem ida                        303560 non-null object
Cidade - Origem ida                    310816 non-null object
País - Destino ida                     310816 non-null object
UF - Destino ida                       301096 non-null object
Cidade - Destino ida                   310816 non-null object
País - Origem volta                    23485 non-null object
UF - Origem volta                      20492 non-null object
Cidade - Origem volta                  23485 non-null object
Pais - Destino volta                   23485 non-null object
UF - Destino volta                     22903 non-null object
Cidade - Destino volta                 23485 non-null object
Valor da passagem           

# Conversão das colunas para os tipos de dados corretos e padronização campos
---

In [9]:
"""
Verificamos valores nulos e tipos de dados com o método info() do dataframe.
"""

df_passagens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310816 entries, 0 to 310815
Data columns (total 16 columns):
Identificador do processo de viagem    310816 non-null int64
Meio de transporte                     310816 non-null object
País - Origem ida                      310816 non-null object
UF - Origem ida                        303560 non-null object
Cidade - Origem ida                    310816 non-null object
País - Destino ida                     310816 non-null object
UF - Destino ida                       301096 non-null object
Cidade - Destino ida                   310816 non-null object
País - Origem volta                    23485 non-null object
UF - Origem volta                      20492 non-null object
Cidade - Origem volta                  23485 non-null object
Pais - Destino volta                   23485 non-null object
UF - Destino volta                     22903 non-null object
Cidade - Destino volta                 23485 non-null object
Valor da passagem           

In [10]:
"""
A conversão pode ser feita utilizando o método apply da série que deseja converter.
"""
def converter_numero(valor):
    try:
        return float(valor.replace('.', '').replace(',', '.'))
    except:
        return np.nan

df_passagens_1 = df_passagens.copy()
df_passagens_1['Valor da passagem'] = ___
df_passagens_1['Taxa de serviço'] = ___

In [11]:
df_passagens_1[['Valor da passagem', 'Taxa de serviço']].head()
df_passagens_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310816 entries, 0 to 310815
Data columns (total 16 columns):
Identificador do processo de viagem    310816 non-null int64
Meio de transporte                     310816 non-null object
País - Origem ida                      310816 non-null object
UF - Origem ida                        303560 non-null object
Cidade - Origem ida                    310816 non-null object
País - Destino ida                     310816 non-null object
UF - Destino ida                       301096 non-null object
Cidade - Destino ida                   310816 non-null object
País - Origem volta                    23485 non-null object
UF - Origem volta                      20492 non-null object
Cidade - Origem volta                  23485 non-null object
Pais - Destino volta                   23485 non-null object
UF - Destino volta                     22903 non-null object
Cidade - Destino volta                 23485 non-null object
Valor da passagem           

In [12]:
"""
Padronização das strings é interessante para que a ausência ou a presença de acentos não interfira em agrupamentos.
"""

from libs.texto import TratamentoTexto
def tratar_texto(valor):
    v = valor
    
    # somente realiza o tratamento se o texto não for nulo
    if v:
        # remove acentuação
        v = TratamentoTexto.remover_acentuacao(v)
        # converte para letras maiúsculas
        v = v.upper()
        
    return v

colunas = ['Meio de transporte',
       'País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida',
       'País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida',
       'País - Origem volta', 'UF - Origem volta', 'Cidade - Origem volta',
       'Pais - Destino volta', 'UF - Destino volta', 'Cidade - Destino volta']

# padronize as colunas da lista acima no dataframe df_passagens_1
___ 

df_passagens_1.head()

,Identificador do processo de viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço
0,14046485,Rodoviário,Brasil,Ceará,Fortaleza,Brasil,Ceará,Sobral,None,None,None,None,None,None,14046485.00,14046485.00
1,14046485,Rodoviário,Brasil,Ceará,Sobral,Brasil,Ceará,Fortaleza,None,None,None,None,None,None,14046485.00,14046485.00
2,14166390,Aéreo,Brasil,Paraná,Londrina,Brasil,Mato Grosso,Sinop,None,None,None,None,None,None,14166390.00,14166390.00
3,14166390,Aéreo,Brasil,Mato Grosso,Sinop,Brasil,Paraná,Londrina,None,None,None,None,None,None,14166390.00,14166390.00
4,14201345,Aéreo,Brasil,Paraná,Curitiba,Brasil,Rio Grande do Sul,Porto Alegre,None,None,None,None,None,None,14201345.00,14201345.00


# Você já está pensando em responder as questões?

Não é uma boa idéia na maioria dos casos.

In [13]:
colunas_gb = ['País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida',
       'País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida']

colunas_selecao = ['País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida',
       'País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida', 'Valor da passagem']

df_passagens_1[colunas_selecao].groupby(colunas_gb, as_index=False).agg(['count', 'mean', 'median']).sort_values(('Valor da passagem', 'count'), ascending=False).head(10)

ValueError: no results

# Entendimento e organização dos dados

Vamos olhar as variáveis de interesse separadamente.

## Identificador do processo de viagem

In [ ]:
plt.rcParams['figure.figsize'] = (16.5,6)
df_passagens_1['Identificador do processo de viagem'].value_counts().value_counts().plot.bar()
plt.xlabel('Quantidade de Trechos por ID de Viagem')
plt.ylabel('Quantidade de Registros')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
'''
Exemplo de uma viagem com 3 trechos
'''
df_passagens_1[ df_passagens_1['Identificador do processo de viagem'] == 14756123 ]

In [ ]:
'''
Exemplo de uma viagem com 5 trechos
'''
df_passagens_1[ df_passagens_1['Identificador do processo de viagem'] == 14825056 ]

In [ ]:
'''
Exemplo de uma viagem com 14 trechos
'''
df_passagens_1[ df_passagens_1['Identificador do processo de viagem'] == 15190575 ]

In [ ]:
df_passagens_1.describe(include='all')

In [ ]:
"""
Vamos criar uma coluna que identifica se o trecho faz parte de uma viagem internacional ou não
"""

# vamos marcar trechos que são de origem ou destino internacional
df_passagens_1['Trecho Internacional'] = \
    (df_passagens_1['País - Origem ida'] != 'BRASIL') | (df_passagens_1['País - Destino ida'] != 'BRASIL') \
    | ( df_passagens_1['País - Origem volta'].notnull() & ((df_passagens_1['País - Origem volta'] != 'BRASIL') | (df_passagens_1['Pais - Destino volta'] != 'BRASIL'))) \

df_passagens_1[df_passagens_1['Trecho Internacional']].head()

In [ ]:
# quais viagens possuem ao menos um trecho internacional
df_viagens_internacionais = \
    df_passagens_1[['Identificador do processo de viagem','Trecho Internacional']]\
    .groupby('Identificador do processo de viagem', as_index=False)\
    .max()

df_viagens_internacionais.head()

In [ ]:
# quais processos de viagem possuem trechos internacionais
df_viagens_internacionais = df_viagens_internacionais.rename(columns={'Trecho Internacional': 'Viagem Internacional'})
df_viagens_internacionais.head()

In [ ]:
df_passagens_2 = pd.merge(df_passagens_1, df_viagens_internacionais, how='left', 
                          left_on='Identificador do processo de viagem', right_on='Identificador do processo de viagem')
df_passagens_2.head()

In [ ]:
df_passagens_2[df_passagens_2['Identificador do processo de viagem'] == 14342418]


In [ ]:
df_passagens_g = df_passagens_2[df_passagens_2['Valor da passagem'] > 0.001] 
plt.rcParams['figure.figsize'] = 16,10
ax = sns.boxplot(y=df_passagens_g['Valor da passagem'], x=df_passagens_g['Trecho Internacional'].apply(lambda x: 'Internacional' if x else 'Nacional'))
ax.set_yscale('log')
ax.plot(x=[-1000, 1000], y=[2000,2000], color='red', linewidth=2, markersize=12)
plt.grid(True)
plt.show()

### Observações

* Número de trechos interfere no valor das passagens?
* Trechos domésticos e viagens internacionais possuem valor superior aos dos trechos domésticos em viagens nacionais?
* Trechos com valores muito baixos. Como isso é possível?

## Meio de transporte
---

In [ ]:
df_passagens_1['Meio de transporte'].value_counts().to_frame()

In [ ]:
plt.rcParams["figure.figsize"] = 16, 4
ax = df_passagens_1['Meio de transporte'].value_counts().plot.barh()
ax.set_xscale('log')
plt.grid(True)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 16,7
plt.rcParams['figure.dpi'] = 90

df_passagens_g = df_passagens_2[df_passagens_2['Valor da passagem']>0]

sns.boxplot(df_passagens_g['Meio de transporte'], df_passagens_g['Valor da passagem'], order=df_passagens_g['Meio de transporte'].drop_duplicates().sort_values())
plt.yscale('log')
plt.ylim = [0, 150000]
plt.grid(True)
plt.show()

### Observações

* Trechos aéreos acompanhados de outras modalidades possuem valor diferenciado?
* Foco nos trechos aéreos

## Taxa de Serviço
---

In [ ]:
"""
Muitos registros zerados.
""" 
df_passagens_1['Taxa de serviço'].value_counts().head()

In [ ]:
df_passagens_1['Taxa de serviço'].describe().to_frame()

In [ ]:
df_passagens_g = df_passagens_2[df_passagens_2['Taxa de serviço']>0]
plt.rcParams['figure.figsize'] = 16,8
sns.boxplot(x='Meio de transporte', y='Taxa de serviço', data=df_passagens_g, order=df_passagens_g['Meio de transporte'].drop_duplicates().sort_values())
plt.yscale('log')
plt.grid(True)
plt.show()

### Observações

* Campo com poucas observações relevantes.
* Talvez seja interessante investigar os outliers com valores altos.
* Para o propósito levantado inicialmente, este campo não é necessário.

## Origem, Destino, Ida e Volta

In [ ]:
"""
Ocorrência da mesma variável em mais de uma coluna no dataset.
"""

df_passagens_2.head()

In [ ]:
"""
Vamos organizar.
"""

df_passagens_3 = df_passagens_2.copy()


In [ ]:
df_passagens_3[df_passagens_3['País - Origem volta'].notnull()].shape

In [ ]:
"""
Vamos quebrar as linhas que possuem os trachos de ida e volta.
"""

df_passagens_3['Tipo Trecho'] = 'IDA'
df_passagens_3['Tipo Compra'] = 'SEPARADA'

voltas = []

for idx, df in df_passagens_3[df_passagens_3['País - Origem volta'].notnull()].iterrows():
    valor_passagem = df_passagens_3.at[idx, 'Valor da passagem'] / 2
    
    df_passagens_3.at[idx, 'Valor da passagem'] = valor_passagem
    df_passagens_3.at[idx, 'Tipo Compra'] = 'CONJUNTA'
    
    for ic in range(8,14):
        df_passagens_3.iat[idx, ic] = None
    
    df['Tipo Trecho'] = 'VOLTA'
    df['Tipo Compra'] = 'CONJUNTA'
    df['Valor da passagem'] = valor_passagem
    
    df['País - Origem ida'] = df['País - Origem volta']
    df['UF - Origem ida'] = df['UF - Origem volta']
    df['Cidade - Origem ida'] = df['Cidade - Origem volta']
    df['País - Destino ida'] = df['Pais - Destino volta']
    df['UF - Destino ida'] = df['UF - Destino volta']
    df['Cidade - Destino ida'] = df['Cidade - Destino volta']
    
    df['País - Origem volta'] = None
    df['UF - Origem volta'] = None
    df['Cidade - Origem volta'] = None
    df['Pais - Destino volta'] = None
    df['UF - Destino volta'] = None
    df['Cidade - Destino volta'] = None    
    
    voltas.append(df)
    
    

In [ ]:
df_voltas = pd.concat(voltas, ignore_index=False, axis=1).T
df_voltas.head()

In [ ]:
df_passagens_4 = pd.concat([df_passagens_3, df_voltas])
df_passagens_4.head()

In [ ]:
df_passagens_4 = df_passagens_4.sort_values(['Identificador do processo de viagem', 'Tipo Trecho']).reset_index(drop=True)
df_passagens_4.loc[10:11]

In [ ]:
df_passagens_5 = df_passagens_4.drop(['País - Origem volta', 'UF - Origem volta', 'Cidade - Origem volta',
       'Pais - Destino volta', 'UF - Destino volta', 'Cidade - Destino volta'], axis=1)
df_passagens_5.head()

In [ ]:
"""
Vamos utilizar apenas a sigla do estado.
"""
df_ufs = pd.read_csv('./dados/lista_ufs.csv', sep=';', encoding='latin1')
df_ufs['UF'] = df_ufs['UF'].apply(tratar_texto)

df_ufs.head()

In [ ]:
df_passagens_6 = pd.merge(df_passagens_5, df_ufs, left_on='UF - Origem ida', right_on='UF', how='left')
df_passagens_6.head()

In [ ]:
df_passagens_6 = pd.merge(df_passagens_6, df_ufs, left_on='UF - Destino ida', right_on='UF', how='left')

In [ ]:
df_passagens_6.head()

In [ ]:
"""
Vamos concatenar Cidade e UF em uma única coluna
"""
df_passagens_7 = df_passagens_6.copy()
df_passagens_7['Origem'] = df_passagens_7['Cidade - Origem ida'] + '-' + df_passagens_7['SG_UF_x'].apply(lambda x: x if pd.notnull(x) else '')
df_passagens_7['Destino'] = df_passagens_7['Cidade - Destino ida'] + '-' + df_passagens_7['SG_UF_y'].apply(lambda x: x if pd.notnull(x) else '')
df_passagens_7.head()

In [ ]:
"""
Selecionamos apenas as colunas desejadas e renomeamos elas
"""
df_passagens_7 = df_passagens_7[['Identificador do processo de viagem', 'Origem', 'Destino', 'Valor da passagem', 'Taxa de serviço', 'Trecho Internacional', 'Viagem Internacional', 'Tipo Trecho', 'Tipo Compra', 'Meio de transporte']]
df_passagens_7.columns = ['ID_VIAGEM', 'ORIGEM', 'DESTINO', 'VALOR', 'TAXA', 'TRECHO_INT', 'VIAGEM_INT', 'TIPO_TRECHO', 'TIPO_COMPRA', 'MEIO_TRANSPORTE']
df_passagens_7['VIAGEM_INT'] = df_passagens_7['VIAGEM_INT'].apply(lambda x: 1 if x else 0)
df_passagens_7['TRECHO_INT'] = df_passagens_7['TRECHO_INT'].apply(lambda x: 1 if x else 0)
df_passagens_7.head()

In [ ]:
df_passagens_7.info()

In [ ]:
"""
Fazemos novamente um tratamento de tipos de dados para as colunas
"""
df_passagens_7['VALOR'] = df_passagens_7['VALOR'].astype(np.float64)
df_passagens_7['TAXA'] = df_passagens_7['TAXA'].astype(np.float64)

df_passagens_7['ORIGEM'] = df_passagens_7['ORIGEM'].astype('category')
df_passagens_7['DESTINO'] = df_passagens_7['DESTINO'].astype('category')

df_passagens_7['TIPO_TRECHO'] = df_passagens_7['TIPO_TRECHO'].astype('category')
df_passagens_7['TIPO_COMPRA'] = df_passagens_7['TIPO_COMPRA'].astype('category')

df_passagens_7.head()

In [ ]:
"""
É possível ver que não existem mais registros com informações nulas e os tipos de dados estão adequados para cada situação.
"""
df_passagens_7.info()

In [ ]:
"""
Vamos eliminar todas as viagens internacionais.
Vamos também manter apenas o meio de transporte AEREO
"""
df_passagens_8 = df_passagens_7[df_passagens_7['VIAGEM_INT'] == False]
df_passagens_8 = df_passagens_8[df_passagens_8['MEIO_TRANSPORTE'] == 'AEREO'].reset_index(drop=True)
df_passagens_8.info()

In [ ]:
df_passagens_8[['ORIGEM','DESTINO']]\
    .groupby(['ORIGEM','DESTINO'])\
    .size().reset_index(name='TOTAL').sort_values('TOTAL', ascending=False).head(10)

In [ ]:
"""
Vamos calcular a quantidade de ocorrências, média, mediana e desvio padrão por trecho
"""



In [ ]:
filtro_a = (df_passagens_7['ORIGEM'] == 'RIO DE JANEIRO-RJ') & (df_passagens_7['DESTINO'] == 'BRASILIA-DF')
filtro_b = (df_passagens_7['ORIGEM'] == 'BRASILIA-DF') & (df_passagens_7['DESTINO'] == 'RIO DE JANEIRO-RJ')

df_passagens_g = df_passagens_7[ (filtro_a | filtro_b) ].copy()

df_passagens_g['ORIGEM'] = df_passagens_g['ORIGEM'].astype(str)

ax = sns.boxplot(x='VALOR', y='ORIGEM', data=df_passagens_g, hue='TIPO_COMPRA')

ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(100))
plt.grid(True)
plt.show()

In [ ]:
df_passagens_g.describe()

In [ ]:
"""
Desenhe um histograma com os valores encontrados para o trecho mais frequente. (Considere ida ou volta)
"""


In [ ]:
"""
Identifique os outliers utilizando as marcações do boxplot.

IQR = q3 - q1
limite inferior = q1 - 1.5*IQR
limite superior = q3 + 1.5*IQR
"""


In [ ]:
"""
Utilize o dataframe df_viagem para incluir as informações de Órgão Solicitante e Motivo da Viagem
"""
df_viagem = pd.read_csv('dados/c04_passagens/2018_Viagem.csv.zip', sep=';', encoding='iso-8859-1', error_bad_lines=False, quotechar="\"")
df_viagem.head()

In [ ]:
"""
Identifique os órgãos com mais ocorrências entre os outliers.
"""

___
__Material produzido para o curso__:
* Introdução à Análise de Dados com Python

__Autor__:
* Fernando Sola Pereira

__Revisão__:
* 1.1